In [ ]:
%%writefile MyDataSet.py
import torch
import torchvision
import numpy as np
import os
from PIL import Image
class MyDataset(torch.utils.data.Dataset):
    # for Airplanes, Cars and Ships dataset
    # train or test folders further have subfolders containing images
    # for each category
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.all_image_paths = []
        self.all_labels = []
        for planecarship_dir in os.listdir(data_dir): # each category folder
            category_path = os.path.join(data_dir, planecarship_dir)
            if not os.path.isdir(category_path):
                continue
            if planecarship_dir == "airplanes":
                label = 0
            elif planecarship_dir == "cars":
                label = 1
            elif planecarship_dir == "ships":
                label = 2
            image_paths = [os.path.join(category_path, f) for f in
            os.listdir(category_path) if f.endswith('.jpg')]
            # list of image filenames for a category, e.g., airplane, car, or ship
            labels = [label for i in range(len(image_paths))]
            self.all_image_paths += image_paths
            self.all_labels += labels
        self.num_classes = len(set(self.all_labels))
    def __len__(self):
        return len(self.all_image_paths)
    def __getitem__(self, index):
        img_path = self.all_image_paths[index]
        label = self.all_labels[index]
        img = Image.open(img_path).convert('RGB')
        if self.transform is not None:
            img = self.transform(img)
        return img, label

Overwriting MyDataSet.py


In [ ]:
%%writefile Utils.py
import torch
from MyDataSet import MyDataset
import torchvision
import matplotlib.pyplot as plt
def get_train_loader(data_dir, batch_size, transform=None):
    dataset = MyDataset(data_dir, transform=transform)
    data_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=4,
    pin_memory=True,
    drop_last=True
    )
    return data_loader
def get_test_loader(data_dir, batch_size, transform=None):
    dataset = MyDataset(data_dir, transform=transform)
    data_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=4,
    pin_memory=True
    )
    return data_loader
def plot_images(images, labels):
    # normalise=True below shifts [-1,1] to [0,1]
    img_grid = torchvision.utils.make_grid(images, nrow=4, normalize=True)
    np_img = img_grid.numpy().transpose(1,2,0) # pytorch has the order, c,w,h
    # to be able to view an image, we need to change the order and
    # put it in width, height, color order
    plt.imshow(np_img)
    plt.show()

Overwriting Utils.py


In [ ]:
%%writefile NetworkLinear.py
import torch
import torch.nn as nn
import torch.nn.functional as F
class NetworkLinear(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(3*224*224, 100)
    self.fc2 = nn.Linear(100, 3)
    self.sm = nn.Softmax(dim=1)
  def forward(self, x):
    x = x.view(-1, 3*224*224)
    x = F.relu(self.fc1(x))
    x= self.sm(self.fc2(x)) # softmax activation on final layer
    return x

Overwriting NetworkLinear.py


In [ ]:
%%writefile NetworkCNN.py
import torch
import torch.nn as nn
import torch.nn.functional as F
class NetworkCNN(nn.Module):
  def __init__(self):
      super().__init__()
      self.conv1 = nn.Conv2d(3, 32, 5) # input is color image with 3 channels
      # 32 is the number of feature maps and the kernel size is 5x5
      self.pool = nn.MaxPool2d(2,2)
      # maxpool will be used multiple times, but defined once
      # in_channels = 32 because self.conv1 output is 32 channels
      self.conv2 = nn.Conv2d(32,6,5)
      # 53*53 comes from the dimension of the last conv layer
      self.fc1 = nn.Linear(6*53*53, 100)
      self.fc2 = nn.Linear(100, 3)
      self.sm = nn.Softmax(dim=1)
  def forward(self, x):
      x = self.pool(F.relu(self.conv1(x)))
      x = self.pool(F.relu(self.conv2(x)))
      x = x.view(-1, 6*53*53)
      x = F.relu(self.fc1(x))
      x = self.sm(self.fc2(x)) # softmax activation on final layer
      return x

Overwriting NetworkCNN.py


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp "/content/drive/MyDrive/Colab Notebooks/Assignment1_1233424/Dataset_PlanesCarsShips.zip" .
import zipfile
with zipfile.ZipFile("Dataset_PlanesCarsShips.zip", 'r') as zip_ref:
    zip_ref.extractall("Dataset_PlanesCarsShips")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls /content/Dataset_PlanesCarsShips

Dataset_PlanesCarsShips


In [ ]:
!mv /content/Dataset_PlanesCarsShips/Dataset_PlanesCarsShips /content/Dataset_PlanesCarsShips_fixed

mv: cannot move '/content/Dataset_PlanesCarsShips/Dataset_PlanesCarsShips' to '/content/Dataset_PlanesCarsShips_fixed/Dataset_PlanesCarsShips': Directory not empty


In [ ]:
!ls /content/Dataset_PlanesCarsShips_fixed/train

airplanes  cars  ships


In [ ]:
%%writefile SimpleClassfication.py
import sys
from torchvision import transforms
import torch
import Utils
from NetworkCNN import NetworkCNN
import torch.optim as optim
from NetworkLinear import NetworkLinear

def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    data_dir_train = "/content/Dataset_PlanesCarsShips_fixed/train"
    data_dir_test = "/content/Dataset_PlanesCarsShips_fixed/test"
    image_transforms = {
    "train": transforms.Compose([
      transforms.Resize((224, 224)),
      transforms.ToTensor(),
      transforms.Normalize([0.5, 0.5, 0.5],[0.5, 0.5, 0.5])]),
    "test": transforms.Compose([
      transforms.Resize((224, 224)),
      transforms.ToTensor(),
      transforms.Normalize([0.5, 0.5, 0.5],
      [0.5, 0.5, 0.5])
      ])
    }
    batch_size = 16
    num_epochs = 25
    train_loader = Utils.get_train_loader(data_dir_train, batch_size,
    image_transforms["train"])
    test_loader = Utils.get_test_loader(data_dir_test, batch_size,
    image_transforms["test"])
    train_iter = iter(train_loader)
    images, labels = next(train_iter) # get a batch of data e.g., 16x3x224x224
    print(images[0].shape)
    Utils.plot_images(images,labels) # plot images
    net = NetworkLinear() # create the simple linear model
    net = NetworkCNN() # create the CNN model
    loss_criterion = torch.nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    running_loss = 0
    print_freq = 100
    for epoch in range(num_epochs):
        for i, data in enumerate(train_loader):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = net(inputs) # forward pass
            loss = loss_criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            #if i % print_freq == print_freq-1:
        print('epoch:',epoch, i+1, running_loss/print_freq)
        running_loss = 0
    total = 0 # keeps track of how many images we have processed
    correct = 0 # keeps track of how many correct images our net predicts
    with torch.no_grad():
      for i, data in enumerate(test_loader):
          images, labels = data
          outputs = net(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size()[0]
          correct += (predicted == labels).sum().item()
    print("Accuracy: ", correct/total)
if __name__ == "__main__":
    sys.exit(int(main() or 0))

Writing SimpleClassfication.py


In [ ]:
!python3 SimpleClassfication.py

Traceback (most recent call last):
  File "/content/SimpleClassfication.py", line 4, in <module>
    import Utils
ModuleNotFoundError: No module named 'Utils'
